In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [22]:
#Determine the years I'm interested in
years = [2019, 2020, 2021, 2022, 2023, 2024, 2025]

#I create an empty data frame
data_frame = []

#read_csv for every season using a for loop
for year in years:
    new_df = pd.read_csv(f"cap_season{year}.csv")
    #assign a year column to indicate the season
    new_df["Year"] = int(year)
    #put all of the dfs together
    data_frame.append(new_df)

#put it all into a df called cap_data
cap_data = pd.concat(data_frame)

cap_data.head(15)

,RANK,TEAM,RECORD,PLAYERS\nACTIVE,AVG AGE\nTEAM,TOTAL CAP\nALLOCATIONS,CAP SPACE\nALL,ACTIVE\n53-MAN,RESERVES\nIR/PUP/NFI/SUSP,DEAD\nCAP,Year
0,1,IND,7-9-0,53,25.7,"$196,100,572","$41,745,481","$142,403,804","$29,183,425","$24,497,343",2019
1,2,CLE,6-10-0,53,25.2,"$212,753,859","$32,328,127","$156,151,812","$24,322,553","$32,279,494",2019
2,3,PHI,9-7-0,52,25.9,"$170,362,399","$28,538,697","$118,556,337","$30,019,958","$21,682,104",2019
3,4,BUF,10-6-0,53,26.9,"$174,804,987","$25,621,283","$152,820,449","$5,049,084","$16,935,454",2019
4,5,KC,12-4-0,53,26.7,"$166,507,002","$23,408,500","$135,179,512","$10,374,764","$20,952,726",2019
5,6,MIA,5-11-0,54,25.4,"$175,757,955","$21,380,539","$60,872,168","$47,954,138","$66,915,649",2019
6,7,TEN,9-7-0,52,26.1,"$197,086,185","$20,789,952","$152,774,763","$30,452,561","$13,858,861",2019
7,8,OAK,7-9-0,51,25.6,"$173,145,802","$19,778,721","$119,270,203","$30,142,965","$23,644,634",2019
8,9,DAL,8-8-0,52,26.6,"$182,367,508","$18,360,084","$148,849,800","$23,186,160","$10,331,548",2019
9,10,DET,3-12-1,53,25.8,"$176,627,499","$17,783,369","$93,511,403","$67,955,813","$15,128,283",2019


In [ ]:
#Use a map to rename all of the teams by their mascot
team_map = {
    "ARI": "Cardinals",
    "ATL": "Falcons",
    "BAL": "Ravens",
    "BUF": "Bills",
    "CAR": "Panthers",
    "CHI": "Bears",
    "CIN": "Bengals",
    "CLE": "Browns",
    "DAL": "Cowboys",
    "DEN": "Broncos",
    "DET": "Lions",
    "GB": "Packers",
    "HOU": "Texans",
    "IND": "Colts",
    "JAX": "Jaguars",
    "KC": "Chiefs",
    "LV": "Raiders",
    "LAC": "Chargers",
    "LAR": "Rams",
    "MIA": "Dolphins",
    "MIN": "Vikings",
    "NE": "Patriots",
    "NO": "Saints",
    "NYG": "Giants",
    "NYJ": "Jets",
    "PHI": "Eagles",
    "PIT": "Steelers",
    "SF": "49ers",
    "SEA": "Seahawks",
    "TB": "Buccaneers",
    "TEN": "Titans",
    "WAS": "Commanders"
}

#Assign my mapped out names to the dataframe
cap_data["TEAM"] = cap_data["TEAM"].map(team_map)

#Rename all of the columns to more df friendly titles
cap_data = cap_data.rename(columns={
    "TEAM": "Team",
    "AVG AGE\nTEAM": "avg_age",
    "ACTIVE\n53-MAN": "active_roster_spending",
    "DEAD\nCAP": "dead_cap",
    "TOTAL CAP\nALLOCATIONS": "total_cap"
})

#select only the variables I'm interested in evaluating
cap_data = cap_data[["Team", "avg_age", "total_cap", "active_roster_spending", "dead_cap", "Year"]]
cap_data.head(15)


#I need to replace the $ and , using regex and the replace function, then set type to integer for the following columns
cap_data["total_cap"] = cap_data["total_cap"].str.replace(r"[$,]", "", regex=True)
cap_data["total_cap"] = cap_data["total_cap"].astype(int)

cap_data["active_roster_spending"] = cap_data["active_roster_spending"].str.replace(r"[$,]", "", regex=True)
cap_data["active_roster_spending"] = cap_data["active_roster_spending"].astype(int)

cap_data["dead_cap"] = cap_data["dead_cap"].str.replace(r"[$,]", "", regex=True)
cap_data["dead_cap"] = cap_data["dead_cap"].astype(int)

In [24]:
#Using downloaded csv files to create season dataframe and adding a year variable
season_df = []

#for loop with each year
for year in years:
    #reads csv files by year
    new_df = pd.read_csv(f"season{year}.csv")
    #adds an integer variable type
    new_df["Year"] = int(year)
    season_df.append(new_df)

#puts all data into one df
season_df = pd.concat(season_df)


In [17]:
season_df.tail(32)

,Tm,W,L,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS,Year
0,New England Patriots*,8,2,0.0,0.800,265,192,73,7.3,-4.3,3.0,2.7,0.3,2025
1,Buffalo Bills+,6,3,0.0,0.667,248,197,51,5.7,-3.0,2.7,3.7,-1.0,2025
2,Miami Dolphins,3,7,0.0,0.300,210,256,-46,-4.6,-0.7,-5.3,-2.9,-2.4,2025
3,New York Jets,2,7,0.0,0.222,195,241,-46,-5.1,-3.7,-8.8,-4.0,-4.8,2025
4,Pittsburgh Steelers*,5,4,0.0,0.556,212,220,-8,-0.9,-0.7,-1.6,-0.7,-0.9,2025
5,Baltimore Ravens,4,5,0.0,0.444,229,235,-6,-0.7,2.0,1.4,3.7,-2.3,2025
6,Cincinnati Bengals,3,6,0.0,0.333,216,300,-84,-9.3,-1.5,-10.8,0.5,-11.3,2025
7,Cleveland Browns,2,7,0.0,0.222,146,211,-65,-7.2,-1.4,-8.6,-8.9,0.3,2025
8,Indianapolis Colts*,8,2,0.0,0.800,321,206,115,11.5,-2.2,9.3,9.2,0.0,2025
9,Jacksonville Jaguars+,5,4,0.0,0.556,205,220,-15,-1.7,3.0,1.4,2.1,-0.8,2025


In [25]:
#cleaning season df

#This code assigns a column named Team where only the last element of the team name is taken
#to do this I used .str.split and str[-1] to get the last string element
season_df["Team"] = season_df["Tm"].str.split().str[-1]
#str.strip was used to get rid of playoff clinching indicators
season_df["Team"] = season_df["Team"].str.strip("*+")

#Created a new column titled win_pct
season_df["win_pct"] = season_df["W-L%"]

#Created a win - loss differential variable to use for exploratory data analysis
season_df["win_loss_diff"] = season_df["W"] - season_df["L"]

#Change the T value to an integer type and make NA values 0
season_df["T"] = pd.to_numeric(season_df["T"], errors="coerce").fillna(0).astype(int)

season_df = season_df[["Team", "W", "L", "T", "win_pct", "win_loss_diff", "PF", "Year"]]


In [74]:
season_df.head(5)

,Team,W,L,T,win_pct,win_loss_diff,PF,Year
0,Patriots,12,4,0,0.750,8,420.0,2019
1,Bills,10,6,0,0.625,4,314.0,2019
2,Jets,7,9,0,0.438,-2,276.0,2019
3,Dolphins,5,11,0,0.313,-6,306.0,2019
4,Ravens,14,2,0,0.875,12,531.0,2019


In [26]:
#Combine the dataframe and create a wrangled/cleaned csv file

#I used pd.merge I merged each observation by Team and Year. 
nfl_cap_win_data = pd.merge(season_df, cap_data, on = ["Team", "Year"], how = "inner")

nfl_cap_win_data.head(20)

,Team,W,L,T,win_pct,win_loss_diff,PF,Year,avg_age,total_cap,active_roster_spending,dead_cap
0,Patriots,12,4,0,0.750,8,420,2019,27.6,192501047,156633955,25721180
1,Bills,10,6,0,0.625,4,314,2019,26.9,174804987,152820449,16935454
2,Jets,7,9,0,0.438,-2,276,2019,26.0,188464903,96678757,27627375
3,Dolphins,5,11,0,0.313,-6,306,2019,25.4,175757955,60872168,66915649
4,Ravens,14,2,0,0.875,12,531,2019,26.8,190028527,139521776,30289731
5,Steelers,8,8,0,0.500,0,289,2019,25.7,206404929,129754417,34440395
6,Browns,6,10,0,0.375,-4,335,2019,25.2,212753859,156151812,32279494
7,Bengals,2,14,0,0.125,-12,279,2019,25.9,184954951,132358691,13714635
8,Texans,10,6,0,0.625,4,378,2019,26.6,198106556,145821737,27378439
9,Titans,9,7,0,0.563,2,402,2019,26.1,197086185,152774763,13858861


In [27]:
nfl_cap_win_data.to_csv("nfl_cap_data.csv")